In [3]:
import pandas as pd
import scipy.sparse as sparse
import csv


import implicit

# Load the data
raw_data = pd.read_csv("./train_Capillary_4/train.csv")[['UserId','productid','Quantity']]
test_data = pd.read_csv("test_nFNPSyV.csv")

In [4]:
raw_data.head()

,UserId,productid,Quantity
0,18075,12322648,1
1,6820,12371370,1
2,6820,12973004,1
3,6820,12657560,1
4,6820,11659914,1


In [5]:
# Drop NaN columns
data = raw_data.dropna()
data = data.copy()

In [6]:
sparse_item_user = sparse.csr_matrix((data['Quantity'].astype(float), (data['productid'], data['UserId'])))

In [10]:
type(sparse_item_user)

scipy.sparse.csr.csr_matrix

In [11]:
sparse_user_item = sparse.csr_matrix((data['Quantity'].astype(float), (data['UserId'], data['productid'])))

In [12]:
# Initialize the als model and fit it using the sparse item-user matrix
model = implicit.als.AlternatingLeastSquares(factors=70, regularization=0.6, iterations=20)

In [ ]:
# Calculate the confidence by multiplying it by alpha value.
alpha_val = 10
data_conf = (sparse_item_user * alpha_val).astype('double')

# Fit the model
model.fit(data_conf)

In [ ]:
# Create recommendations for all users
fields=['UserId','product_list']
filename = 'submission_als.csv'
with open(filename,'a') as f:
    writer = csv.writer(f)
    writer.writerow(fields)
    userids = test_data['UserId'].drop_duplicates().values.tolist()
    for user in userids:
        products = []
        scores = []
        results = []
        results.append(user)
        recommended = model.recommend(user, sparse_user_item)

        for item in recommended:
            idx, score = item
            products.append(idx)
            scores.append(score)
        results.append(products)
        #print products
        #print scores
        writer.writerow(results)